# 🔍 LLMs — Response Objects & Unified Interfaces

This notebook covers three topics:

1. **Azure OpenAI** — anatomy of the `ChatCompletion` response object
2. **Anthropic Claude** — anatomy of the `Message` response object and differences from OpenAI
3. **LangChain** — unified interface for multiple LLM providers


## 1. Azure OpenAI — Response Object Deep Dive

Every call to the Chat Completions API returns a rich response object packed with
metadata, usage statistics, and — of course — the generated content itself.

**🎯 What You'll Learn:**
- The full anatomy of a `ChatCompletion` response object
- How to navigate choices, messages, and usage statistics
- How to inspect detailed token-usage breakdowns (reasoning, audio, etc.)


In [ ]:
import textwrap

from dotenv import load_dotenv
from openai.lib.azure import AzureOpenAI
from openai.types.chat import (
    ChatCompletionSystemMessageParam,
    ChatCompletionUserMessageParam,
)

load_dotenv(override=True)

True

In [ ]:
azure_openai_client = AzureOpenAI()

### Response Object Anatomy

The Chat Completions API returns a `ChatCompletion` object that contains:
* **id** — unique identifier of this completion request
* **model** — the model that actually served the request
* **choices[]** — list of generated replies (usually one)
  * **message** — the assistant's message with role and content
* **usage** — token counts (prompt, completion, total)


In [ ]:
chat_messages = [
    ChatCompletionSystemMessageParam(role="system",
                                     content="You are a helpful assistant who explains concepts clearly and concisely."),
    ChatCompletionUserMessageParam(role="user", content="Why is the sky blue?"),
]

completion_response = azure_openai_client.chat.completions.create(
    model="gpt-5-nano",
    messages=chat_messages,
)

assistant_answer = completion_response.choices[0].message.content.strip()
print(assistant_answer)

### Field-by-Field Exploration

In [ ]:
response_object_type = type(completion_response)
print(f"1. Response Object Type:\n   {response_object_type}\n")

response_id = completion_response.id
print(f"2. Response ID:\n   {response_id}\n")

model_used = completion_response.model
print(f"3. Model Used:\n   {model_used}\n")

full_response_repr = completion_response
print(f"4. Full Response Object:\n   {full_response_repr}\n")

choices_array = completion_response.choices
print(f"5. Choices Array:\n   {choices_array}\n")

first_choice = completion_response.choices[0]
print(f"6. First Choice Object:\n   {first_choice}\n")

message_object = first_choice.message
print(f"7. Message Object:\n   {message_object}\n")

message_content = message_object.content
print(f"8. Message Content:\n   {message_content}\n")

usage_summary = completion_response.usage
print(f"9. Usage Statistics:\n   {usage_summary}\n")

In [ ]:
completion_response.__dict__

### Detailed Usage Statistics

Token usage is critical for cost management and performance monitoring.

The usage object always includes:
* **prompt_tokens** — tokens consumed by the input (system + user messages)
* **completion_tokens** — tokens generated by the model
* **total_tokens** — sum of the above

Some models also expose `completion_tokens_details` with granular breakdowns
such as `reasoning_tokens` and `audio_tokens`.

In [ ]:
if completion_response.usage:
    prompt_token_count = completion_response.usage.prompt_tokens
    completion_token_count = completion_response.usage.completion_tokens
    total_token_count = completion_response.usage.total_tokens

    print(f"Prompt tokens:     {prompt_token_count}")
    print(f"Completion tokens: {completion_token_count}")
    print(f"Total tokens:      {total_token_count}")

    if hasattr(completion_response.usage, "completion_tokens_details"):
        token_details = completion_response.usage.completion_tokens_details
        if token_details:
            print(f"Reasoning tokens:  {token_details.reasoning_tokens}")
            print(f"Audio tokens:      {token_details.audio_tokens}")

print("\n💡 The response object contains rich metadata that can be used")
print("   for monitoring, logging, and understanding API usage patterns.")


## 2. Anthropic Claude — Response Object Deep Dive

Every call to the Claude Messages API returns a structured response object with
metadata, usage statistics, and the generated content itself.

**🎯 What You'll Learn:**
- The full anatomy of a Claude `Message` response object
- How to navigate content blocks, text, and usage statistics
- Key structural differences between the Claude and OpenAI APIs

In [ ]:
import os

import anthropic
from anthropic.types import MessageParam, TextBlockParam

In [ ]:
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
anthropic_client = anthropic.Anthropic(api_key=anthropic_api_key)

### Response Object Anatomy

The Claude Messages API returns a `Message` object that contains:
* **id** — unique identifier of this message request
* **model** — the model that actually served the request
* **content[]** — list of content blocks (usually one `TextBlock`)
  * **.text** — the actual generated text
* **usage** — token counts (`input_tokens`, `output_tokens`)

In [ ]:
user_message = MessageParam(
    role="user",
    content=[TextBlockParam(type="text", text="Why is the sky blue?")],
)

claude_response = anthropic_client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1000,
    temperature=1.0,
    system="You are a friendly assistant answering users' questions. You respond in corporate slang with many anglicisms.",
    messages=[user_message],
)

assistant_answer_text = claude_response.content[0].text
print(textwrap.fill(assistant_answer_text, width=80))

### Field-by-Field Exploration

In [ ]:
response_object_type = type(claude_response)
print(f"1. Response Object Type:\n   {response_object_type}\n")

full_response_repr = claude_response
print(f"2. Full Response Object:\n   {full_response_repr}\n")

model_used = claude_response.model
print(f"3. Model Used:\n   {model_used}\n")

content_array = claude_response.content
print(f"4. Content Array:\n   {content_array}\n")

first_content_block = claude_response.content[0]
print(f"5. First Content Block:\n   {first_content_block}\n")

first_content_block_text = claude_response.content[0].text
print(f"6. First Content Block Text:\n   {first_content_block_text}\n")

usage_summary = claude_response.usage
print(f"7. Usage Statistics:\n   {usage_summary}\n")

### Detailed Usage Statistics

Token usage is critical for cost management and performance monitoring.

The Claude usage object includes:
* **input_tokens** — tokens consumed by the input (system + user messages)
* **output_tokens** — tokens generated by the model

Note: Unlike OpenAI, Claude does not provide a `total_tokens` field,
so we compute it ourselves.

In [ ]:
if claude_response.usage:
    input_token_count = claude_response.usage.input_tokens
    output_token_count = claude_response.usage.output_tokens
    total_token_count = input_token_count + output_token_count

    print(f"Input tokens:  {input_token_count}")
    print(f"Output tokens: {output_token_count}")
    print(f"Total tokens:  {total_token_count}")

### Claude vs OpenAI — Structural Differences

Claude and OpenAI APIs share similar concepts but differ in structure.
Understanding these differences is key when switching between providers.

| Aspect         | Claude                           | OpenAI                                |
|----------------|----------------------------------|---------------------------------------|
| Method         | `messages.create()`              | `chat.completions.create()`           |
| Content path   | `content[0].text`                | `choices[0].message.content`          |
| System message | separate `system` parameter      | part of `messages` array              |
| Usage fields   | `input_tokens` / `output_tokens` | `prompt_tokens` / `completion_tokens` |


## 3. LangChain — Unified Interface for Multiple Providers

LangChain provides a single abstraction layer so you can swap LLM providers
without rewriting application logic.

**🎯 What You'll Learn:**
- How LangChain wraps Azure OpenAI and Claude behind a common interface
- How to use Ollama for local model inference via LangChain
- Why a unified abstraction matters for swapping models painlessly

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import AzureChatOpenAI
from pydantic import SecretStr

### Azure OpenAI — via LangChain

LangChain wraps Azure OpenAI behind a unified `ChatModel` interface.
Calling `.invoke(question)` returns an `AIMessage` — same shape regardless of provider.

In [ ]:
azure_langchain_llm = AzureChatOpenAI(model="gpt-4o-mini")
sky_question = "Why is the sky blue?"
azure_langchain_response = azure_langchain_llm.invoke(sky_question)
print(azure_langchain_response.content)

### Anthropic Claude — via LangChain

The same `.invoke()` call works identically for Claude, making it trivial
to swap providers without touching application logic.

In [ ]:
anthropic_api_key_secret = SecretStr(os.environ["ANTHROPIC_API_KEY"])
claude_langchain_llm = ChatAnthropic(
    model_name="claude-3-5-sonnet-20241022",
    api_key=anthropic_api_key_secret,
    timeout=30,
    stop=["\n\nHuman:", "\n\nAssistant:"],
)
sky_question = "Why is the sky blue?"
claude_langchain_response = claude_langchain_llm.invoke(sky_question)
print(claude_langchain_response.content)

## Zadanie

Przetestować połączenie z lokalnymi modelami LLM poprzez Ollama przy użyciu biblioteki LangChain.

- 📥 Instalacja Ollama: https://ollama.com/download/linux
- 🏷️ Przykładowy model: https://ollama.com/library/gemma3:1b

📚 Dokumentacja LangChain: https://docs.langchain.com/oss/python/langchain/overview
